In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250404'

## Processing Raw Test Cases

In [14]:
colorder = ['Device', "Section", "소분류", '사전조건', '테스트 항목', '기대결과']
df_tests = pd.read_csv('data/KOCOWA_3.5_test_cases.csv')[colorder].rename(columns={"Section": "Scenario"}).fillna("")

# Create one-hot encoded device columns
df_device_dummies = pd.get_dummies(df_tests["Device"], prefix="Device")

# Combine with the rest of the test case columns
df_tests_per_device = pd.concat([df_device_dummies, df_tests.drop(columns=["Device"])], axis=1)

df_tests_per_device = df_tests_per_device.rename(columns={'Device_AOS': "Android Mobile", 'Device_AndroidTV': "Android TV", 'Device_AppleTV': "Apple TV",
       'Device_Connected TV': "Connected TV", 'Device_FireTV': "Fire TV", 'Device_Mobile': "Mobile", 'Device_ROKU': "Roku",
       'Device_Tablet': "Tablet", 'Device_WEB': "Web", 'Device_iOS': "Apple Mobile"})

for index, row in df_tests_per_device.iterrows():
       if row.Mobile == True:
              df_tests_per_device.at[index, "Android Mobile"] = True
              df_tests_per_device.at[index, "Apple Mobile"] = True

df_tests_per_device = df_tests_per_device.drop(columns=["Mobile", "Tablet"])

print(len(list(df_tests_per_device.sort_values(by=["Scenario","소분류","테스트 항목", "기대결과"])["테스트 항목"].unique())))

df_tests_per_device.sort_values(by=["Scenario","소분류","테스트 항목", "기대결과"])

690


,Android Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web,Apple Mobile,Scenario,소분류,사전조건,테스트 항목,기대결과
580,False,False,False,False,False,False,True,False,Category List Area,,1. PLAN Page 진입,Title Text 확인,"""Subscription"" text 출력된다"
588,False,False,False,False,False,False,True,False,Category List Area>Basic Plan Option,,1. PLAN Page 진입,Bullet point 확인,"Benefit text 앞 ""●"" 형태의 글머리 기호 노출된다"
593,False,False,False,False,False,False,True,False,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,1. PLAN Page 진입,item Hover 시 UI 확인,item 흰색으로 강조되어 노출된다
596,False,False,False,False,False,False,True,False,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,1. PLAN Page 진입\n2. Basic Montlhy Subscription...,item 결제 시 UI 확인,해당 Subscription item Dimmed 처리되어 노출된다
595,False,False,False,False,False,False,True,False,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,1. PLAN Page 진입,item 선택 시 동작 확인,Checkout Page 진입된다
...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page>Body,,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,Next Button 확인,"""Next"" Button 노출된다"
950,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page>Body,,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,User Email address 확인,User Email Address 출력된다
951,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page>Body,,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,sub text 확인,"""An authentication email will arrive to your e..."
949,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page>Body,,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,title text 확인,"""Completed to create your account"" text 출력된다"


## Processing Billing Gateway Validation Test Cases to Append

## Pivot Data to Use Device Columns as Boolean Flags

In [15]:
test_case_keys = ["Scenario","소분류","테스트 항목", "기대결과"]

device_columns = ['Android Mobile', "Apple Mobile", 'Android TV', 'Apple TV', 'Connected TV', 'Fire TV', 'Roku', 'Web']

df_tests_grouped = df_tests_per_device.groupby(test_case_keys)[device_columns].max().reset_index()
df_tests_grouped = df_tests_grouped.sort_values(by=test_case_keys).reset_index(drop=True)
print(df_tests_grouped.Scenario.unique())
print(df_tests_per_device.Scenario.unique())
print(len(list(df_tests_grouped.index)))

df_tests_grouped

['Category List Area' 'Category List Area>Basic Plan Option'
 'Category List Area>Basic Plan Option>Annual Subscription'
 'Category List Area>Basic Plan Option>Monthly Subscription'
 'Category List Area>Bottom Area' 'Category List Area>Header Area'
 'Category List Area>Premium Plan Option'
 'Category List Area>Premium Plan Option>Annual Subscription'
 'Category List Area>Premium Plan Option>Monthly Subscription'
 'Checkout Page' 'Checkout Page>Billing Information Area'
 'Checkout Page>Bottom Area' 'Checkout Page>Payment Summary Area'
 'Chromecast>Casting Action' 'Chromecast>Casting Player Screen'
 'Chromecast>Casting Player Screen>Player UI' 'Curation Area'
 'Curation Area>Hovered Poster image'
 'Input PIN Code Webpage>Functional Area'
 'Input PIN Code Webpage>Header Area'
 'Input PIN Code Webpage>input box Area'
 'Input PIN Code Webpage>input box Area>IME'
 'Input PIN Code Webpage>input box Area>Validation'
 'KOCOWA ID Sign In>Email Input Area'
 'KOCOWA ID Sign In>Email Input Area>Val

,Scenario,소분류,테스트 항목,기대결과,Android Mobile,Apple Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web
0,Category List Area,,Title Text 확인,"""Subscription"" text 출력된다",False,False,False,False,False,False,False,True
1,Category List Area>Basic Plan Option,,Bullet point 확인,"Benefit text 앞 ""●"" 형태의 글머리 기호 노출된다",False,False,False,False,False,False,False,True
2,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,item Hover 시 UI 확인,item 흰색으로 강조되어 노출된다,False,False,False,False,False,False,False,True
3,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,item 결제 시 UI 확인,해당 Subscription item Dimmed 처리되어 노출된다,False,False,False,False,False,False,False,True
4,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,item 선택 시 동작 확인,Checkout Page 진입된다,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1162,Welcome to KOCOWA Page>Body,,Next Button 확인,"""Next"" Button 노출된다",True,True,False,False,False,False,False,False
1163,Welcome to KOCOWA Page>Body,,User Email address 확인,User Email Address 출력된다,True,True,False,False,False,False,False,False
1164,Welcome to KOCOWA Page>Body,,sub text 확인,"""An authentication email will arrive to your e...",True,True,False,False,False,False,False,False
1165,Welcome to KOCOWA Page>Body,,title text 확인,"""Completed to create your account"" text 출력된다",True,True,False,False,False,False,False,False


In [18]:
df_device_dummies.columns

Index(['Device_AOS', 'Device_AndroidTV', 'Device_AppleTV',
       'Device_Connected TV', 'Device_FireTV', 'Device_Mobile', 'Device_ROKU',
       'Device_Tablet', 'Device_WEB', 'Device_iOS'],
      dtype='object')

In [10]:
list_of_devices = list(df_tests.Device.unique())

for device in list_of_devices:
    num_cases = len(list(df_tests[df_tests.Device == device]["테스트 항목"].unique()))
    print(f"{device} has {num_cases} number of test cases")

Connected TV has 185 number of test cases
AppleTV has 53 number of test cases
ROKU has 23 number of test cases
AndroidTV has 13 number of test cases
FireTV has 1 number of test cases
WEB has 314 number of test cases
Mobile has 245 number of test cases
iOS has 4 number of test cases
AOS has 2 number of test cases
Tablet has 1 number of test cases


In [ ]:
for index, row in df_tests.iterrows():
    # 대분류 0 -> 중분류
    if row["Region/Category"] == 0:
        replacement = "All Regions"
        df_tests.at[index, "Region/Category"] = replacement
    
    # Language Settings to be under All Regions
    if row["Region/Category"] == "Language Setting":
        df_tests.at[index, "소분류"] = "Language Setting"
        df_tests.at[index, "Region/Category"] = "All Regions"
    
    # if 중분류 == multi profile page, All Regions and move 대분류 -> 소분류
    if row["중분류"] == "Multi Profile Page":
        task = row['Region/Category']
        df_tests.at[index, "소분류"] = task
        df_tests.at[index, 'Region/Category'] = "All Regions"
        
    # if 중분류 == social/sso, All Regions and move 대분류 -> 소분류
    if row["중분류"] == "Social Sign in" or row["중분류"] == "Social Sign up" or row["중분류"] == "SSO":
        platform = row['Region/Category']
        df_tests.at[index, "소분류"] = platform
        df_tests.at[index, 'Region/Category'] = "All Regions"
    
    # Consolidate PLAN crosses, put them under All Regions, PLAN Cross to 중분류
    if row["Region/Category"] == "PLAN Cross":
        start_plan = row.중분류
        end_plan = row.소분류
        df_tests.at[index, "소분류"] = f"{start_plan} > {end_plan}"
        df_tests.at[index, "중분류"] = "PLAN Cross"
        df_tests.at[index, "Region/Category"] = "All Regions"
    
    # Compile all Plan Changes
    if row["중분류"] == "PLAN Downgrade":
        df_tests.at[index, "소분류"] = "Downgrade"
        df_tests.at[index, "중분류"] = "PLAN Change (Up/Downgrade)"
    elif row["중분류"] == "PLAN Upgrade":
        df_tests.at[index, "소분류"] = "Upgrade"
        df_tests.at[index, "중분류"] = "PLAN Change (Up/Downgrade)"
    elif row["중분류"] == "PLAN Change":
        df_tests.at[index, "중분류"] = "PLAN Change (Up/Downgrade)"
        
# df_tests[df_tests["Region/Category"] == "All Regions"]
df_regional_consolidation = df_tests[~df_tests["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom"])]

for index, row in df_regional_consolidation.iterrows():
    if row["소분류"] not in ['Basic Month Single Pass', 'Basic Annual Single Pass', 'Premium Month Single Pass', 'Premium Annual Single Pass']:
        df_regional_consolidation.at[index, "Region/Category"] = "All Regions"
df_regional_consolidation.drop_duplicates()


,Region/Category,중분류,소분류,사전조건,테스트 항목,테스트 스텝,기대결과
8,All Regions,Sign in,0,1. KOCOWA Webpage 진입\n2. 미로그인 상태,Sign in 기능 확인,Sign in 시도,Sign in 진행된다
9,All Regions,Sign Up,0,1. KOCOWA Webpage 진입\n2. 미로그인 상태,Sign up 기능 확인,Sign up 시도,Sign up 진행된다
10,All Regions,Sign Out,0,1. KOCOWA Webpage 진입\n2. 로그인 상태,Sign out 기능 확인,Sign out 시도,Sign out 진행된다
11,All Regions,Sign Out,0,1. KOCOWA App 실행\n2. Sign In\n3. Sign Out,재로그인 가능 확인,해당 계정으로 Sign In,Sign in 진행된다
12,All Regions,Find Password,0,1. KOCOWA Webpage 진입\n2. Sign in page 진입\n3. F...,Find Password 기능 확인,Find Password 시도,Reset Password 이메일 수신된다
...,...,...,...,...,...,...,...
2104,All Regions,Resume PLAN,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,구독 갱신 일자 노출 확인,구독 갱신 일자 노출 확인,"[MM/DD/YYYY, HH:MM] [AM/PM] [대륙/도시] 노출된다"
2105,All Regions,Resume PLAN,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,Resume Button 선택 시 기능 확인,Resume Button 선택,구독 재개된다
2106,All Regions,Resume PLAN,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 갱신 text 출력 확인,Resume Button 선택,Next Billing Date : text 출력된다
2107,All Regions,Resume PLAN,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 갱신 일자 노출 확인,구독 갱신 일자 노출 확인,"[MM/DD/YYYY, HH:MM] [AM/PM] [대륙/도시] 노출된다"


In [ ]:
# df_tests[~df_tests["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom", "Brazil"])].sort_values(by=["Region/Category", "중분류"])
# df_tests[df_tests.중분류 == 0]
# df_tests[df_tests["중분류"] == "PLAN Cross"]
# df_tests[~df_tests["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom"])]

In [ ]:
df_regional = df_tests[df_tests["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom", "New Zealand", "Brazil"])]
f"Count of test cases by region: US-{df_regional[df_regional["Region/Category"] == "US"]["테스트 항목"].count()}, Argentina-{df_regional[df_regional["Region/Category"] == "Argentina"]["테스트 항목"].count()}, Spain-{df_regional[df_regional["Region/Category"] == "Spain"]["테스트 항목"].count()}, United Kingdom-{df_regional[df_regional["Region/Category"] == "United Kingdom"]["테스트 항목"].count()}, Australia-{df_regional[df_regional["Region/Category"] == "Australia"]["테스트 항목"].count()}, Brazil-{df_regional[df_regional["Region/Category"] == "Brazil"]["테스트 항목"].count()}"

'Count of test cases by region: US-411, Argentina-386, Spain-55, United Kingdom-375, Australia-377, Brazil-493'

In [166]:
df_regional_consolidation["Region/Category"].unique()

array(['All Regions', 'Brazil'], dtype=object)

In [167]:
df_regional_consolidation.중분류.unique()

array(['Sign in', 'Sign Up', 'Sign Out', 'Find Password',
       'Social Sign in', 'Social Sign up', 'Multi Profile Page', 'Home',
       'Content Playble', 'KOAS', 'PLAN Page', 'Checkout Page',
       'Checkout Page\n(Coupon)', 'PLAN Purchasing', 'Cancel PLAN',
       'Renewal Subscription', 'PLAN Change', 'SSO', 'QR Sign in',
       'PLAN Cross', 'Resume PLAN'], dtype=object)

In [168]:
middle_layer_values = df_regional_consolidation.중분류.unique()
for value in middle_layer_values:
    lower_layer_values = df_regional_consolidation[df_regional_consolidation.중분류 == value].소분류.unique()
    print(f"소분류 values for {value}: {lower_layer_values}")
    print()

소분류 values for Sign in: [0]

소분류 values for Sign Up: [0]

소분류 values for Sign Out: [0]

소분류 values for Find Password: [0]

소분류 values for Social Sign in: ['Facebook' 'Google' 'Apple' 'ODK']

소분류 values for Social Sign up: ['Facebook' 'Google' 'Apple' 'ODK']

소분류 values for Multi Profile Page: ['Add Profile' 'Edit Profile' 'Delete Profile' 'Profile Change'
 'PIN Locked']

소분류 values for Home: ['Language Setting']

소분류 values for Content Playble: [0 'KOBACO Policy']

소분류 values for KOAS: [0 'SSO' 'Profile Change' 'KOCOWA ID']

소분류 values for PLAN Page: ['Basic Monthly' 'Basic Annual' 'Premium Monthly' 'Premium Annual'
 'Basic Month Single Pass' 'Basic Annual Single Pass'
 'Premium Month Single Pass' 'Premium Annual Single Pass']

소분류 values for Checkout Page: [0 'Basic Monthly' 'Basic Annual' 'Premium Monthly' 'Premium Annual'
 'Basic Month Single Pass' 'Basic Annual Single Pass'
 'Premium Month Single Pass' 'Premium Annual Single Pass' 'Basic Annuals']

소분류 values for Checkout Page
(Cou

In [5]:
df_regional_consolidation.to_csv(f"data/{today}_processed_billing_test_cases.csv", index=False)